In [1]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


In [2]:
/* specify schema for clickstream data */
import org.apache.spark.sql.types._

val clickdataSchema = StructType(Array(
      StructField("eventId", LongType, false),
      StructField("eventType", StringType, false),
      StructField("timestamp", StringType, false),
      StructField("ipaddress", StringType, false),
      StructField("sessionId", StringType, false),
      StructField("userId", StringType, false),
      StructField("pageUrl", StringType, false),
      StructField("browser", StringType, false)))

In [3]:
/** IBM Event Store imports and connection information */
import sys.process._
import scala.concurrent.{Await, Future}
import scala.concurrent.duration.Duration
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import com.ibm.event.catalog.TableSchema
import com.ibm.event.oltp.EventContext
import com.ibm.event.example.DataGenerator
import com.ibm.event.common.ConfigurationReader
import com.ibm.event.oltp.InsertResult
// import com.ibm.bluspark.example.BluSparkUtil
ConfigurationReader.setConnectionEndpoints("192.168.0.101:1100")

In [4]:
/** Connect to the IBM Event Store */
import java.io.File
import org.apache.log4j.{Level, LogManager, Logger}
import org.apache.spark._
import org.apache.spark.sql.ibm.event.EventSession
val sqlContext = new EventSession(spark.sparkContext, "TESTDB")
val table = sqlContext.loadEventTable("ClickStreamTable")
table.registerTempTable("ClickStreamTable")

val clickStreamDF = sqlContext.sql("select * from ClickStreamTable")
clickStreamDF.show(5)

+-----------+---------+----------+------------+-----------------+------+--------------------+-------+
|    eventId|eventType| timestamp|   ipaddress|        sessionId|userId|             pageUrl|browser|
+-----------+---------+----------+------------+-----------------+------+--------------------+-------+
|20170522901| pageView|1496311260|169.34.56.78|y20170522a4499u21|ceaton|  /www.cybershop.com| Chrome|
|20170522902| pageView|1496311320|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|
|20170522903| pageView|1496311440|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|
|20170522904| pageView|1496311500|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|
|20170522905| pageView|1496311560|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|
+-----------+---------+----------+------------+-----------------+------+--------------------+-------+
only showing top 5 rows



### Analyze Clickstream data

In [5]:
/* Calculate time_on_page */
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

val timestamp = clickStreamDF("timestamp")
val next_timestamp = lead(timestamp, 1).over(Window.orderBy(timestamp))
val clickStreamWithTimeDF = clickStreamDF.withColumn("time", next_timestamp.cast(LongType) - timestamp.cast(LongType))
clickStreamWithTimeDF.show(5)

+-----------+---------+----------+------------+-----------------+------+--------------------+-------+----+
|    eventId|eventType| timestamp|   ipaddress|        sessionId|userId|             pageUrl|browser|time|
+-----------+---------+----------+------------+-----------------+------+--------------------+-------+----+
|20170522901| pageView|1496311260|169.34.56.78|y20170522a4499u21|ceaton|  /www.cybershop.com| Chrome|  60|
|20170522902| pageView|1496311320|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome| 120|
|20170522903| pageView|1496311440|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|  60|
|20170522904| pageView|1496311500|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|  60|
|20170522905| pageView|1496311560|169.34.56.78|y20170522a4499u21|ceaton|/estore?product_l...| Chrome|  60|
+-----------+---------+----------+------------+-----------------+------+--------------------+-------+----+
only showing top 5 rows



In [6]:
/* Calculate date on page_view */
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

clickStreamWithTimeDF.registerTempTable("tempData")
val clickStreamWithDateTimeDF = sqlContext.sql("select eventId, eventType, cast(from_unixtime(timestamp) as date), ipaddress,sessionId,userId,pageUrl,browser,time from tempData").
                            withColumnRenamed("CAST(from_unixtime(CAST(timestamp AS BIGINT), yyyy-MM-dd HH:mm:ss) AS DATE)","date")

//clickStreamWithDateTimeDF.show(5)

/* build aggregated web metrics from clickstream data*/
clickStreamWithDateTimeDF.registerTempTable("ClickData")
val clicksDF = sqlContext.sql("select pageURL, count(*) as page_hits, sum(time) as total_time from ClickData where eventType='pageView' group by pageURL")
clicksDF.show(5,false)

+-----------------------------------------------------------------------------+---------+----------+
|pageURL                                                                      |page_hits|total_time|
+-----------------------------------------------------------------------------+---------+----------+
|/www.cybershop.com                                                           |9        |600       |
|/estore?product_line=smartphones&action=catalog                              |13       |1260      |
|/estore?product_line=smartphones&product=A-phone&action=details              |7        |540       |
|/estore?product_line=smartphones&product=A-phone&feature=color&action=details|5        |660       |
|/estore?product_line=smartphones&product=S-phone&action=details              |5        |600       |
+-----------------------------------------------------------------------------+---------+----------+
only showing top 5 rows



In [7]:
/* build aggregated web metrics by product_line, products and feature browses */
clicksDF.registerTempTable("WebMetricsData")
val webMetricsDF = sqlContext.sql("select * from WebMetricsData")
webMetricsDF.show(5)

+--------------------+---------+----------+
|             pageURL|page_hits|total_time|
+--------------------+---------+----------+
|  /www.cybershop.com|        9|       600|
|/estore?product_l...|       13|      1260|
|/estore?product_l...|        7|       540|
|/estore?product_l...|        5|       660|
|/estore?product_l...|        5|       600|
+--------------------+---------+----------+
only showing top 5 rows



In [8]:
/* build aggregated web metrics by product_line, products and feature browses */
clicksDF.registerTempTable("WebMetricsDataTest")
val metricsQuery = """select parse_URL(pageURL,'QUERY','product_line') as product_line, 
                        Coalesce(parse_URL(pageURL,'QUERY','action'),'') as action,
                        Coalesce(parse_URL(pageURL,'QUERY','product'),'') as product, 
                        Coalesce(parse_URL(pageURL,'QUERY','feature'),'') as feature, page_hits, total_time from WebMetricsData"""
val metricsQuery2 = """select parse_URL(pageURL,'QUERY','product_line') as product_line, 
                        parse_URL(pageURL,'QUERY','action') as action,
                        parse_URL(pageURL,'QUERY','product') as product, 
                        from WebMetricsData"""
val metricsQuery3 = "select parse_URL(pageURL,'QUERY','product_line') as product_line from WebMetricsDataTest"
val webMetricsDF3 = sqlContext.sql(metricsQuery3).filter($"product_line".isNotNull).sort($"product_line".desc)
webMetricsDF3.show(5)
val webMetricsDF = sqlContext.sql(metricsQuery).filter($"product_line".isNotNull).sort($"product_line".desc)
webMetricsDF.show(5)

+------------+
|product_line|
+------------+
|  videogames|
|  videogames|
| smartphones|
| smartphones|
| smartphones|
+------------+
only showing top 5 rows

+------------+-------+-------+---------+---------+----------+
|product_line| action|product|  feature|page_hits|total_time|
+------------+-------+-------+---------+---------+----------+
|  videogames|details| W-game|         |        1|       120|
|  videogames|catalog|       |         |        6|      4680|
| smartphones|details|A-phone|processor|        2|       120|
| smartphones|details|A-phone|         |        7|       540|
| smartphones|details|A-phone|    color|        5|       660|
+------------+-------+-------+---------+---------+----------+
only showing top 5 rows



### Aggregated Web Metrics for All Product Lines

In [9]:
/* build aggregated web metrics per product line */
val productlineMetrics = webMetricsDF.select("product_line","page_hits","total_time").
    groupBy("product_line").agg(sum("page_hits"), sum("total_time")).
    withColumnRenamed("sum(page_hits)","page_hits").
    withColumnRenamed("sum(total_time)","total_time")

productlineMetrics.sort($"page_hits".desc).show(10)


+--------------+---------+----------+
|  product_line|page_hits|total_time|
+--------------+---------+----------+
|   smartphones|       58|      6360|
|     computers|       16|      3720|
|    videogames|        7|      4800|
|    appliances|        5|      2220|
|   hometheater|        4|       840|
|    headphones|        4|       960|
|carelectronics|        2|       420|
|       cameras|        2|       360|
+--------------+---------+----------+



In [10]:
%%brunel data('productlineMetrics') 
bar x(product_line) y(page_hits) tooltip(#all)color(product_line)legends(none) axes(x:'product lines',y:'page views')sort(page_hits)|
stack polar bar  y(total_time) color(product_line)label(product_line) legends(none) tooltip("time on page (sec): ",total_time)sort(page_hits) 
 :: width=1000, height=300

### Aggregated Web Metrics for Smart Phones

In [11]:
/* Visualize aggregated page hits and browse time */
val productMetrics = webMetricsDF.select("product_line","product","page_hits","total_time").filter($"action" === "details").filter($"product_line" === "smartphones").
    groupBy("product_line","product").agg(sum("page_hits"), sum("total_time")).
    withColumnRenamed("sum(page_hits)","page_hits").
    withColumnRenamed("sum(total_time)","total_time")
productMetrics.show()

+------------+-------+---------+----------+
|product_line|product|page_hits|total_time|
+------------+-------+---------+----------+
| smartphones|A-phone|       21|      1920|
| smartphones|S-phone|       12|      1380|
| smartphones|M-phone|        5|       540|
| smartphones|L-phone|        3|       180|
| smartphones|H-phone|        2|       120|
| smartphones|X-phone|        1|       720|
+------------+-------+---------+----------+



In [12]:
%%brunel data('productMetrics') 
bar x(product) y(page_hits) tooltip(page_hits,product)color(product) legends(none) axes(x:'smart phones',y:'page views')sort(page_hits)|
bar x(product) y(total_time) color(product)label(product)tooltip("time on page (sec): ",total_time) legends(none)sort(page_hits)

### Aggregated Web Metrics on Smart Phone Features 

In [13]:
 /*Visualize aggregated page hits for Features */
val featureMetrics = webMetricsDF.select("product","feature","page_hits","total_time").filter($"action" === "details").filter($"product" === "A-phone").
    filter("feature != ''").groupBy("product","feature").agg(sum("page_hits"), sum("total_time")).
    withColumnRenamed("sum(page_hits)","page_hits").
    withColumnRenamed("sum(total_time)","total_time")

featureMetrics.show()

+-------+---------------+---------+----------+
|product|        feature|page_hits|total_time|
+-------+---------------+---------+----------+
|A-phone|          color|        5|       660|
|A-phone|        battery|        1|       120|
|A-phone|      processor|        2|       120|
|A-phone|voice_assistant|        2|       120|
|A-phone|         camera|        3|       180|
+-------+---------------+---------+----------+



In [14]:
%%brunel data('featureMetrics') 
bar x(feature) y(page_hits) tooltip(feature,page_hits)color(feature) legends(none) axes(x:'A-phone features',y:'page views')sort(page_hits)interaction(select)|
stack polar bar  y(total_time) color(feature)label(feature) tooltip("time on page (sec): ",total_time) legends(none)sort(page_hits)opacity(#selection)
:: width=1000, height=300

### Web Metrics for user 'David'

In [15]:
/* get user web metrics from clickstream data */
val userClicksQuery ="""select pageURL,year(date) as year,month(date) as month,weekofyear(date) as week,day(date) as day,
                        count(*) as page_hits, sum(time) as total_time from ClickData where eventType='pageView' and
                        userId='datkins' group by pageURL, date"""
val userClicksDF = sqlContext.sql(userClicksQuery)        
userClicksDF.show(5)

+--------------------+----+-----+----+---+---------+----------+
|             pageURL|year|month|week|day|page_hits|total_time|
+--------------------+----+-----+----+---+---------+----------+
|  /www.cybershop.com|2017|    6|  24| 16|        1|        60|
|/estore?product_l...|2017|    6|  24| 16|        1|       240|
|/estore?product_l...|2017|    6|  24| 16|        1|       240|
|/estore?product_l...|2017|    6|  24| 16|        1|       180|
|/estore?product_l...|2017|    6|  24| 16|        1|       120|
+--------------------+----+-----+----+---+---------+----------+
only showing top 5 rows



In [16]:
/* build user web metrics by product_line, products and feature browses */
userClicksDF.registerTempTable("UserWebMetricsData")
val metricsQuery = """select month,week,day, parse_URL(pageURL,'QUERY','product_line') as product_line, 
                        Coalesce(parse_URL(pageURL,'QUERY','action'),'') as action,
                        Coalesce(parse_URL(pageURL,'QUERY','product'),'') as product, 
                        Coalesce(parse_URL(pageURL,'QUERY','feature'),'') as feature, page_hits, total_time from UserWebMetricsData
                        where year = '2017'"""

val userWebMetricsDF = sqlContext.sql(metricsQuery).filter($"product_line".isNotNull)
userWebMetricsDF.show(5)

+-----+----+---+------------+-------+-------+---------------+---------+----------+
|month|week|day|product_line| action|product|        feature|page_hits|total_time|
+-----+----+---+------------+-------+-------+---------------+---------+----------+
|    6|  24| 16|  headphones|details| A-head|               |        1|       240|
|    6|  24| 16|  headphones|details| B-head|               |        1|       240|
|    6|  24| 16| smartphones|details|M-phone|               |        1|       180|
|    6|  24| 16| smartphones|details|S-phone|               |        1|       120|
|    6|  24| 16| smartphones|details|S-phone|voice_assistant|        1|        60|
+-----+----+---+------------+-------+-------+---------------+---------+----------+
only showing top 5 rows



In [17]:
/* visualize metrics for most recent week */
val weekMetricsDF = userWebMetricsDF.groupBy("day","product_line","action","product","feature","page_hits","total_time").max("week")
weekMetricsDF.show(5)

+---+------------+-------+-------+---------------+---------+----------+---------+
|day|product_line| action|product|        feature|page_hits|total_time|max(week)|
+---+------------+-------+-------+---------------+---------+----------+---------+
| 16|  headphones|details| A-head|               |        1|       240|       24|
| 16|  headphones|details| B-head|               |        1|       240|       24|
| 16| smartphones|details|M-phone|               |        1|       180|       24|
| 16| smartphones|details|S-phone|               |        1|       120|       24|
| 16| smartphones|details|S-phone|voice_assistant|        1|        60|       24|
+---+------------+-------+-------+---------------+---------+----------+---------+
only showing top 5 rows



In [18]:
%%brunel data('weekMetricsDF') 
x(day)y(page_hits) stack bar sum (page_hits) color(product_line) tooltip(#all)axes(x:'Day of Month' ,y:'page views') axes(x:7) interaction(select)  |
stack polar bar  y(total_time) color(product_line)label(product) legends(none) tooltip(#all)opacity(#selection)
:: width=1000, height=300


##### Insight summary from Clickstream Analysis

1. Aggregated web metrics of recent months highlights significant interest in Smart phones with A-phones leading the pack. 
2. User 'David' is a repeat visitor and has explored Smart phones multiple times in recent days along with Computers and Headphones. 


###### sanne0611v4 